# Crystal relaxation

This Notebook follows the workflow for identifying and analyzing bulk crystal structures as is currently being done with the Interatomic Potentials Repository.

**Quick Notes:**

- All input scripts take key-value pairs where the "key"s correspond to the calculation's input parameters.

- Multiple calculations are prepared by specifying multiple values for the same key (on separate lines).

- The special "buildcombos" key accesses predefined functions for generating lists of input parameter values for certain sets of keys.  See documentation for more details.

**Library imports**

In [1]:
# Standard Python libraries
from __future__ import (absolute_import, print_function,
                        division, unicode_literals)
import os

# http://www.numpy.org/
import numpy as np

from IPython.core.display import display, HTML

# https://pandas.pydata.org/
import pandas as pd

from DataModelDict import DataModelDict as DM

# https://github.com/usnistgov/atomman
import atomman.unitconvert as uc

# https://github.com/usnistgov/iprPy
import iprPy
print('iprPy version', iprPy.__version__)

iprPy version 0.8.2


## 0. Access database 

### Load database

In [2]:
database = iprPy.load_database('demo')

## 1. relax_box calculation

This calculation statically relaxes a given system by only adjusting the box dimensions to zero pressure without any internal relaxations, i.e. all atoms retain box-relative positions.

In [3]:
calculation = iprPy.load_calculation('relax_box')
run_directory = iprPy.load_run_directory('demo_1')

### Show calculation's allowed keys

In [4]:
print(calculation.allkeys)

['lammps_command', 'mpi_command', 'length_unit', 'pressure_unit', 'energy_unit', 'force_unit', 'potential_file', 'potential_content', 'potential_dir', 'load_file', 'load_content', 'load_style', 'family', 'load_options', 'symbols', 'box_parameters', 'a_uvw', 'b_uvw', 'c_uvw', 'atomshift', 'sizemults', 'pressure_xx', 'pressure_yy', 'pressure_zz', 'strainrange']


### Write input script

In [5]:
input_script = """
# Commands and executables
lammps_command              lmp_mpi
mpi_command                 

# Build load information based on reference structures
buildcombos                 atomicreference load_file reference

# Specify reference buildcombos limiters (only build for potential listed)
reference_potential_name    1986--Foiles-S-M--Ag--LAMMPS--ipr1

# Build load information from E_vs_r_scan results
buildcombos                 atomicparent load_file parent

# Specify parent buildcombos terms (parent record's style and the load_key to access)
parent_record               calculation_E_vs_r_scan              
parent_load_key             minimum-atomic-system

# System manipulations
a_uvw                      
b_uvw                      
c_uvw                    
atomshift                   
sizemults                   10 10 10

# Units that input/output values are in
length_unit                 
pressure_unit               
energy_unit                 
force_unit                  

# Run parameters
strainrange                 1e-6
"""
with open('input_script.in', 'w') as f:
    f.write(input_script)

### Prepare calculations

In [6]:
with open('input_script.in') as f:
    input_dict = iprPy.input.parse(f, singularkeys=calculation.singularkeys)
    
database.prepare(run_directory, calculation, **input_dict)
database.check_records(calculation.record_style)

In database style local at C:\Users\lmh1\Documents\calculations\ipr\demo :
- 62 of style calculation_relax_box
 - 28 are complete
 - 31 still to run
 - 3 issued errors


### Run calculations

In [7]:
database.runner(run_directory)
database.check_records(calculation.record_style)

Runner started with pid 10608
No simulations left to run
In database style local at C:\Users\lmh1\Documents\calculations\ipr\demo :
- 62 of style calculation_relax_box
 - 56 are complete
 - 0 still to run
 - 6 issued errors


In [8]:
results_df = database.get_records_df(style=calculation.record_style)
for error in np.unique(results_df[results_df.status=='error'].error):
    print(error)

b'Traceback (most recent call last):
  File "calc_relax_box.py", line 458, in <module>
    main(*sys.argv[1:])
  File "calc_relax_box.py", line 54, in main
    strainrange = input_dict[\'strainrange\'])
  File "calc_relax_box.py", line 145, in relax_box
    strainrange=strainrange, cycle=cycle)
  File "calc_relax_box.py", line 364, in calc_cij
    raise RuntimeError(\'Divergence of box dimensions to <= 0\')
RuntimeError: Divergence of box dimensions to <= 0
'
b'Traceback (most recent call last):
  File "calc_relax_box.py", line 458, in <module>
    main(*sys.argv[1:])
  File "calc_relax_box.py", line 54, in main
    strainrange = input_dict[\'strainrange\'])
  File "calc_relax_box.py", line 177, in relax_box
    raise RuntimeError(\'Divergence of box dimensions\')
RuntimeError: Divergence of box dimensions
'
b'Traceback (most recent call last):
  File "calc_relax_box.py", line 458, in <module>
    main(*sys.argv[1:])
  File "calc_relax_box.py", line 54, in main
    strainrange = input_

## 2. relax_dynamic calculation

This calculation dymamically relaxes a given system for a specified number of MD integrations at a specified temperature, pressure, etc.  Here, we are only doing 0 K relaxations.

In [9]:
calculation = iprPy.load_calculation('relax_dynamic')
run_directory = iprPy.load_run_directory('demo_4')

### Write input script

In [10]:
input_script = """
# Commands and executables
lammps_command              lmp_mpi
mpi_command                 c:\\Program Files\\MPICH2\\bin\\mpiexec -localonly 4

# Build load information based on reference structures
buildcombos                 atomicreference load_file reference

# Specify reference buildcombos limiters (only build for potential listed)
reference_potential_name    1986--Foiles-S-M--Ag--LAMMPS--ipr1

# Build load information from E_vs_r_scan results
buildcombos                 atomicparent load_file parent

# Specify parent buildcombos terms (parent record's style and the load_key to access)
parent_record               calculation_E_vs_r_scan              
parent_load_key             minimum-atomic-system

# System manipulations
a_uvw                      
b_uvw                      
c_uvw       
atomshift                   
sizemults                   10 10 10

# Units that input/output values are in
length_unit                 
pressure_unit               
energy_unit                 
force_unit                  

# Run parameters
temperature                 0.0
pressure_xx                 
pressure_yy                 
pressure_zz                 
pressure_xy                 
pressure_xz                 
pressure_yz                 
integrator                  nph+l
thermosteps                 1000
dumpsteps                   
runsteps                    10000
equilsteps                  0
randomseed                  
"""
with open('input_script.in', 'w') as f:
    f.write(input_script)

### Prepare calculations

In [11]:
with open('input_script.in') as f:
    input_dict = iprPy.input.parse(f, singularkeys=calculation.singularkeys)
    
database.prepare(run_directory, calculation, **input_dict)
database.check_records(calculation.record_style)

In database style local at C:\Users\lmh1\Documents\calculations\ipr\demo :
- 62 of style calculation_relax_dynamic
 - 31 are complete
 - 31 still to run
 - 0 issued errors


### Run calculations

In [12]:
database.runner(run_directory)
database.check_records(calculation.record_style)

Runner started with pid 10608
No simulations left to run
In database style local at C:\Users\lmh1\Documents\calculations\ipr\demo :
- 62 of style calculation_relax_dynamic
 - 62 are complete
 - 0 still to run
 - 0 issued errors


In [13]:
results_df = database.get_records_df(style=calculation.record_style)
for error in np.unique(results_df[results_df.status=='error'].error):
    print(error)

## 3. relax_static calculation

This calculation statically relaxes a given system using energy minimizations combined with box dimension relaxations.  Here, we pass in results from both the E_vs_r_scan calculation and the relax_dynamic calculation.

In [14]:
calculation = iprPy.load_calculation('relax_static')
run_directory = iprPy.load_run_directory('demo_1')

### Write input script

In [15]:
input_script = """
# Commands and executables
lammps_command              lmp_mpi
mpi_command                 

# Build load information based on reference structures
buildcombos                 atomicreference load_file reference

# Specify reference buildcombos limiters (only build for potential listed)
reference_potential_name    1986--Foiles-S-M--Ag--LAMMPS--ipr1

# Build load information from E_vs_r_scan results
buildcombos                 atomicparent load_file parent

# Specify parent buildcombos terms (parent record's style and the load_key to access)
parent_record               calculation_E_vs_r_scan              
parent_load_key             minimum-atomic-system

# Build load information from relax_dynamic results
buildcombos                 atomicarchive load_file archive

# Specify archive parent buildcombos terms (parent record's style and the load_key to access)
archive_record              calculation_relax_dynamic
archive_load_key            final-system

# System manipulations
a_uvw                      
b_uvw                      
c_uvw                 
atomshift                   
sizemults                   1 1 1

# Units that input/output values are in
length_unit                 
pressure_unit               
energy_unit                 
force_unit                  

# Run parameters
energytolerance             0.0
forcetolerance              1e-10 eV/angstrom
maxiterations               10000
maxevaluations              100000
maxatommotion               0.01 angstrom
maxcycles                   100
cycletolerance              1e-10
"""
with open('input_script.in', 'w') as f:
    f.write(input_script)

### Prepare calculations

In [16]:
with open('input_script.in') as f:
    input_dict = iprPy.input.parse(f, singularkeys=calculation.singularkeys)
    
database.prepare(run_directory, calculation, **input_dict)
database.check_records(calculation.record_style)

In database style local at C:\Users\lmh1\Documents\calculations\ipr\demo :
- 124 of style calculation_relax_static
 - 54 are complete
 - 62 still to run
 - 8 issued errors


### Run calculations

In [17]:
database.runner(run_directory)
database.check_records(calculation.record_style)

Runner started with pid 10608
No simulations left to run
In database style local at C:\Users\lmh1\Documents\calculations\ipr\demo :
- 124 of style calculation_relax_static
 - 106 are complete
 - 0 still to run
 - 18 issued errors


In [18]:
results_df = database.get_records_df(style=calculation.record_style)
for error in np.unique(results_df[results_df.status=='error'].error):
    print(error)

b'Traceback (most recent call last):
  File "C:\\Users\\lmh1\\AppData\\Local\\Continuum\\anaconda3\\lib\\shutil.py", line 544, in move
    os.rename(src, real_dst)
FileNotFoundError: [WinError 2] The system cannot find the file specified: \'10000.dump\' -> \'relax_static-0.dump\'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "calc_relax_static.py", line 380, in <module>
    main(*sys.argv[1:])
  File "calc_relax_static.py", line 62, in main
    ctol = input_dict[\'cycletolerance\'])
  File "calc_relax_static.py", line 231, in relax_static
    shutil.move(last_dump_file, renamed_dump_file)
  File "C:\\Users\\lmh1\\AppData\\Local\\Continuum\\anaconda3\\lib\\shutil.py", line 558, in move
    copy_function(src, real_dst)
  File "C:\\Users\\lmh1\\AppData\\Local\\Continuum\\anaconda3\\lib\\shutil.py", line 257, in copy2
    copyfile(src, dst, follow_symlinks=follow_symlinks)
  File "C:\\Users\\lmh1\\AppData\\Local\\Continuum\\

## 4. crystal_space_group calculation

This calculation analyzes the space group of a given system.  Here, this is used to determine if the bulk system's structure has transformed.

In [19]:
calculation = iprPy.load_calculation('crystal_space_group')
run_directory = iprPy.load_run_directory('demo_1')

In [20]:
print(calculation.allkeys)

['length_unit', 'pressure_unit', 'energy_unit', 'force_unit', 'load_file', 'load_content', 'load_style', 'family', 'load_options', 'symbols', 'box_parameters', 'symmetryprecision', 'primitivecell', 'idealcell']


### Write input script

In [21]:
input_script = """

# Build load information based on prototype records
buildcombos                 crystalprototype load_file

# Build load information based on reference structures
buildcombos                 atomicreference load_file ref

# Specify reference buildcombos limiters (only build for element sets listed)
ref_elements                Ag

# Build load information from relax_static results
buildcombos                 atomicarchive load_file relax_static

# Specify archive parent buildcombos terms (parent record's style and the load_key to access)
relax_static_record         calculation_relax_static
relax_static_load_key       final-system

# Build load information from relax_box results
buildcombos                 atomicarchive load_file relax_box

# Specify archive parent buildcombos terms (parent record's style and the load_key to access)
relax_box_record            calculation_relax_box
relax_box_load_key          final-system

# Units that input/output values are in
length_unit                 
pressure_unit               
energy_unit                 
force_unit                  

# Run parameters
symmetryprecision           
primitivecell               
idealcell                   
"""
with open('input_script.in', 'w') as f:
    f.write(input_script)

### Prepare calculations

In [22]:
with open('input_script.in') as f:
    input_dict = iprPy.input.parse(f, singularkeys=calculation.singularkeys)
    
database.prepare(run_directory, calculation, **input_dict)
database.check_records(calculation.record_style)

In database style local at C:\Users\lmh1\Documents\calculations\ipr\demo :
- 202 of style calculation_crystal_space_group
 - 122 are complete
 - 80 still to run
 - 0 issued errors


### Run calculations

In [23]:
database.runner(run_directory)
database.check_records(calculation.record_style)

Runner started with pid 10608
No simulations left to run
In database style local at C:\Users\lmh1\Documents\calculations\ipr\demo :
- 202 of style calculation_crystal_space_group
 - 202 are complete
 - 0 still to run
 - 0 issued errors


In [24]:
results_df = database.get_records_df(style=calculation.record_style)
for error in np.unique(results_df[results_df.status=='error'].error):
    print(error)

## 5. Calculation analysis

In [25]:
crystal_match_file = 'reference_prototype_match.csv'

### Load crystal_match_file

In [26]:
ref_proto_match = pd.read_csv(crystal_match_file)

### Retrieve finished calculation results

In [27]:
spg_records = database.get_records_df(style='calculation_crystal_space_group', full=True, flat=False, status='finished')

In [28]:
# Get key lists for relax_* calculations
raw_df = database.get_records_df(style='calculation_relax_box', full=False, flat=True)
try:
    box_keys = raw_df.key.tolist()
except:
    box_keys = []

raw_df = database.get_records_df(style='calculation_relax_static', full=False, flat=True)
try:
    static_keys = raw_df.key.tolist()
except:
    static_keys = []

raw_df = database.get_records_df(style='calculation_relax_dynamic', full=False, flat=True)
try:
    dynamic_keys = raw_df.key.tolist()
except:
    dynamic_keys = []

In [29]:
pot_records = database.get_records_df(style='potential_LAMMPS')

### Identify compositions

In [30]:
iprPy.analysis.assign_composition(spg_records, database)

### Split all spg records into references, prototypes and calculation relaxes

In [31]:
spg_records['record_type'] = 'calc'
spg_records.loc[(spg_records.load_file == spg_records.family + '.poscar'), 'record_type'] = 'reference'
spg_records.loc[(spg_records.load_file == spg_records.family + '.json'), 'record_type'] = 'prototype'

prototype_records = spg_records[spg_records.record_type == 'prototype']
reference_records = spg_records[spg_records.record_type == 'reference']
family_records = spg_records[(spg_records.record_type == 'prototype') | (spg_records.record_type == 'reference')]

calc_records = spg_records[spg_records.record_type == 'calc'].reset_index(drop=True)

In [32]:
calc_records.keys()

Index(['error', 'family', 'idealcell', 'iprPy_version', 'key', 'load_file',
       'load_options', 'load_style', 'pearson_symbol', 'primitivecell',
       'script', 'spacegroup_Schoenflies', 'spacegroup_international',
       'spacegroup_number', 'status', 'symbols', 'symmetryprecision', 'ucell',
       'wykoff_fingerprint', 'composition', 'record_type'],
      dtype='object')

### Analyze calculation results

In [33]:
results = []
for series in calc_records.itertuples():
    results_dict = {}
    
    # Copy over values in series    
    results_dict['calc_key'] = series.key
    results_dict['composition'] = series.composition
    results_dict['family'] = series.family
    results_dict['a'] = series.ucell.box.a
    results_dict['b'] = series.ucell.box.b
    results_dict['c'] = series.ucell.box.c
    results_dict['alpha'] = series.ucell.box.alpha
    results_dict['beta'] = series.ucell.box.beta
    results_dict['gamma'] = series.ucell.box.gamma
    
    # Identify prototype
    try:
        results_dict['prototype'] = ref_proto_match[ref_proto_match.reference==series.family].prototype.values[0]
    except:
        results_dict['prototype'] = series.family
    else:
        if pd.isnull(results_dict['prototype']):
            results_dict['prototype'] = series.family
    
    # Check if structure has transformed relative to reference
    family_series = family_records[family_records.family == series.family].iloc[0]
    results_dict['transformed'] = (not (family_series.spacegroup_number == series.spacegroup_number
                                   and family_series.pearson_symbol == series.pearson_symbol))
    
    # Extract info from parent calculations
    for parent in database.get_parent_records(name=series.key):
        parent_dict = parent.todict()

        if parent_dict['key'] in box_keys:
            results_dict['method'] = 'box'
            results_dict['E_coh'] = parent_dict['E_cohesive']
            results_dict['potential_LAMMPS_key'] = parent_dict['potential_LAMMPS_key']
            continue

        elif parent_dict['key'] in dynamic_keys:
            results_dict['method'] = 'dynamic'
            continue

        elif parent_dict['key'] in static_keys:
            results_dict['method'] = 'static'
            results_dict['E_coh'] = parent_dict['E_cohesive']
            results_dict['potential_LAMMPS_key'] = parent_dict['potential_LAMMPS_key']
    
    pot_record = pot_records[pot_records.key == results_dict['potential_LAMMPS_key']].iloc[0]
    results_dict['potential_id'] = pot_record.pot_id
    results_dict['potential_key'] = pot_record.pot_key
    results_dict['potential_LAMMPS_id'] = pot_record.id
    
    results.append(results_dict)
columns = ['calc_key', 'potential_LAMMPS_key', 'potential_LAMMPS_id', 'potential_key', 'potential_id',
           'composition', 'prototype', 'family', 'method', 'transformed',
           'E_coh', 'a', 'b', 'c', 'alpha', 'beta', 'gamma']
results = pd.DataFrame(results, columns=columns)

### Save raw crystal data per crystal

In [34]:
# Settings
outputpath = 'C:/Users/lmh1/Documents/website/calc_content'
savecolumns = ['calc_key',
               'prototype', 'family', 'method', 'transformed', 
               'E_coh', 'a', 'b', 'c', 'alpha', 'beta', 'gamma']

In [35]:
for implememtation_key in np.unique(results.potential_LAMMPS_key):
    imp_results = results[results.potential_LAMMPS_key == implememtation_key]
    potential = imp_results.iloc[0].potential_id
    implementation = imp_results.iloc[0].potential_LAMMPS_id
    
    contentpath = os.path.join(outputpath, potential, implementation)
    if not os.path.isdir(contentpath):
        os.makedirs(contentpath)
    
    for composition in np.unique(imp_results.composition):
        comp_results = imp_results[imp_results.composition == composition].sort_values('E_coh')
        fstem = 'crystal.' + composition
        
        comp_results[savecolumns].to_csv(os.path.join(contentpath, fstem + '.csv'), index=False)

### Identify unique crystals

In [39]:
unique = pd.DataFrame(columns=results.columns)
for implememtation_key in np.unique(results.potential_LAMMPS_key):
    imp_results = results[results.potential_LAMMPS_key == implememtation_key]
    
    for composition in np.unique(results.composition):
        comp_unique = pd.DataFrame(columns=results.columns)
        comp_results = imp_results[imp_results.composition == composition]
        
        for prototype in np.unique(comp_results.prototype):
            proto_results = comp_results[comp_results.prototype == prototype]
            for method in ['dynamic', 'static', 'box']:
                for i, series in proto_results[(proto_results.method == method)
                                              &(~proto_results.transformed)].iterrows():
                    try:
                        matches = comp_unique[(np.isclose(comp_unique.E_coh, series.E_coh))
                                             &(np.isclose(comp_unique.a, series.a))
                                             &(np.isclose(comp_unique.b, series.b))
                                             &(np.isclose(comp_unique.c, series.c))
                                             &(np.isclose(comp_unique.alpha, series.alpha))
                                             &(np.isclose(comp_unique.beta, series.beta))
                                             &(np.isclose(comp_unique.gamma, series.gamma))]
                    except:
                        matches = []
                    if len(matches) == 0:
                        comp_unique = comp_unique.append(series)
        unique = unique.append(comp_unique)
unique.to_csv('unique_crystals.csv', index=False)

### Add info to PotentialProperties records

This is for generating XML records that the Interatomic Potential Repository uses to automatically build webcontent (done elsewhere).

In [40]:
for implememtation_key in np.unique(results.potential_LAMMPS_key):
    imp_results = results[results.potential_LAMMPS_key == implememtation_key]
    imp_unique = unique[unique.potential_LAMMPS_key == implememtation_key]
    potential_key = imp_results.iloc[0].potential_key
    potential_id = imp_results.iloc[0].potential_id
    implementation_id = imp_results.iloc[0].potential_LAMMPS_id
    
    record_name = 'properties.' + implementation_id
    try:
        record = database.get_record(name=record_name, style='PotentialProperties')
    except:
        new = True
        content = DM()
        content['per-potential-properties'] = DM()
        content['per-potential-properties']['potential'] = DM()
        content['per-potential-properties']['potential']['key'] = potential_key
        content['per-potential-properties']['potential']['id'] = potential_id
        content['per-potential-properties']['implementation'] = DM()
        content['per-potential-properties']['implementation']['key'] = implememtation_key
        content['per-potential-properties']['implementation']['id'] = implementation_id
    else:
        content = DM(record.content)
        new = False
    
    content['per-potential-properties']['crystal-structure'] = model = DM()
    
    # Build prototype-ref-set elements
    for composition in np.unique(imp_results.composition):
        comp_results = imp_results[imp_results.composition == composition]
        for prototype in np.unique(comp_results.prototype):
            proto_results = comp_results[comp_results.prototype == prototype]
            refs = []
            for family in np.unique(proto_results.family):
                if family != prototype:
                    refs.append(family)
            if len(refs) > 0:
                proto_ref_set = DM()
                proto_ref_set['composition'] = composition 
                proto_ref_set['prototype'] = prototype
                for ref in refs:
                    proto_ref_set.append('ref', ref)
                model.append('prototype-ref-set', proto_ref_set)    
    
    # Build crystal elements
    for series in imp_unique.sort_values('E_coh').itertuples():
        crystal = DM()
        crystal['composition'] = series.composition
        crystal['prototype'] = series.prototype
        crystal['method'] = series.method
        crystal['cohesive-energy'] = DM([('value', '%#.4f'%series.E_coh), ('unit', 'eV')])
        crystal['a'] = DM([('value', '%#.4f'%series.a), ('unit', 'angstrom')])
        crystal['b'] = DM([('value', '%#.4f'%series.b), ('unit', 'angstrom')])
        crystal['c'] = DM([('value', '%#.4f'%series.c), ('unit', 'angstrom')])
        crystal['alpha'] = DM([('value', '%#.1f'%series.alpha), ('unit', 'degree')])
        crystal['beta'] = DM([('value', '%#.1f'%series.beta), ('unit', 'degree')])
        crystal['gamma'] = DM([('value', '%#.1f'%series.gamma), ('unit', 'degree')])
        model.append('crystal', crystal)
        
    if new:
        database.add_record(name=record_name, style='PotentialProperties', content=content.xml())
    else:
        database.update_record(record=record, content=content.xml())